In [ ]:
import os
import sys
import yara
from pathlib import Path
from tqdm import tqdm
from multiprocessing import Pool
from datetime import datetime

In [ ]:
if not Path("rules").exists():
    !git clone https://github.com/Yara-Rules/rules

In [ ]:
if not Path("social_rulesets").exists():
    !git clone git@github.com:Koodous/social_rulesets.git

In [ ]:
if not Path("social_rulesets/index.yar").exists():
    with open("social_rulesets/index.yar",'w') as f:
        f.write(f"""/*
Generated by Sam2332
On {datetime.now().date()}
*/\n""")
        for file in Path("social_rulesets/rulesets").iterdir():
            f.write(f'include "./rulesets/{file.name}"\n')

In [ ]:
rules_file = "social_rulesets/index.yar"
rules = yara.compile(rules_file)

In [ ]:
class MalwareResults:
    def __init__(self,file,matches):
        self.file = file
        self.matches = matches
    def __repr__(self):
        return f"{self.file} -- {self.matches}"

In [ ]:
def isMalware(file):
    try:
        if not file.is_file():
            return
        filename = str(file)
        matches = rules.match(filename)
        if len(matches) > 0:
            return MalwareResults(file=file,matches=matches)
    except Exception as e:
        print(e)

In [ ]:
blacklisted_suffix=[
    '.png',
    '.bmp',
    '.jpg',
]
def filename_filter(l_files):
    for file in l_files:
        if file.suffix in blacklisted_suffix:
            continue
        yield file

In [ ]:
for item in Path('phone_sync_dir').iterdir():
    print(f"scanning {item}", end='')
    if item.is_dir():
        #with Pool(15) as p:
        items = list(item.glob('**/*'))
        items = list(filename_filter(items))
        print(f" {len(items)}")
        results =[]
        for item in tqdm(items):
            results.append(isMalware(item))#p.map(isMalware, items[50:])
        results = list(filter(None, results))
        print(len(results))
        for item in results:
            print(item)
            print()
    print()
print('Done')